In [1]:
FREECADPATH = 'C:/Users/Slashito/Documents/FreeCAD_0.19.24219_Win-LPv12.4.3/bin'

import sys
sys.path.append(FREECADPATH)

# Macro Begin: C:\Users\Slashito\AppData\Roaming\FreeCAD\Macro
import FreeCAD as App
import FreeCAD, Sketcher

import FabCAD #Custom Library

nameProject = "FirstSteps"
namePart = "Pieza01"
nameBody = "Cuerpo"

# Crear un nuevo documento en blanco
doc = FabCAD.nuevoDocumento(nameProject)
doc.nuevaPieza(namePart).nuevoCuerpo(nameBody).crearPlano("Planta").crearPlano("Alzado", soporte="XZ_Plane", cuerpo = nameBody).crearPlano("Vista_Lateral", "YZ_Plane", doc.cuerpos[nameBody])
nuevoSketch = doc.nuevoSketch("sketch", doc.cuerpos[nameBody], doc.planos["Planta"])


In [2]:
for llave, item in doc.planos.items(): print(f"{llave}")

Planta
Alzado
Vista_Lateral


In [4]:
nuevoSketch = doc.nuevoSketch("sketch", doc.cuerpos[nameBody], doc.planos["Planta"])

In [5]:
nuevoSketch.crearCirculo([150, 50], 15)

print(nuevoSketch.contGeometria)

puntos = []

puntos.append([47,40])
puntos.append([54,55])
puntos.append([64,33])
puntos.append([69,48])
puntos.append([74,38])
puntos.append([83,60])

nuevoSketch.crearPolilinea(puntos)

print(nuevoSketch.contGeometria)

1
6


In [6]:
sumaX = 0
longitudH = 20
longitudLinea = 10

#Note the use of FreeCAD original library inside the custom library wrap inside doc.base variable 
for linea in range(3):
	nuevoSketch.crearLinea( [ (2 + sumaX) , 2] , [(12 + sumaX), 12] )
	doc.base.getObject("sketch").addConstraint(Sketcher.Constraint('Distance', linea, longitudLinea)) 

	if linea > 0:
		doc.base.getObject("sketch").addConstraint(Sketcher.Constraint('Parallel',  linea -1, linea)) 
		doc.base.getObject("sketch").addConstraint(Sketcher.Constraint('Horizontal', linea - 1, 1, linea, 1)) 		
		doc.base.getObject("sketch").addConstraint(Sketcher.Constraint('DistanceX', linea - 1, 1, linea, 1, App.Units.Quantity('{} mm'.format( longitudH )))) 

	sumaX =sumaX + 20
	longitudLinea += 10

doc.base.getObject("sketch").addConstraint(Sketcher.Constraint('Angle',0,1,-1,2, App.Units.Quantity('135.000000 deg'))) 


13

# Crear ranuras
## Para crear ranuras el metodo interno de la libreria FabCAD hace lo siguiente:

> Primero que se han recibido los dos puntos de inicio y fin de la ranura se calcula su pendiente, esto con la finalidad de obtener los angulos de inicio y fin de los arcos, una vez que se obtiene esta pendiente se le agregan 90 grados para que los arcos sean perpendiculares a la linea recta entre los dos puntos y se colocan los dos puntos de inicio y fin de cada arco, por ultimo segun el radio indicado se coloca otro punto colineal a la linea recta entre los dos puntos y con la longitud del radio especificado, luego estos puntos se bloquean para que queden en la posicion indicada y se dibuja toda la geometria a la que posteriormente se le dará restricciones de posicion

## Ejemplo:
### Se quiere dibujar una ranura entre los puntos [2,1] y [8,5] con un radio de 2mm

> Lo primero que hace el metodo es calcular m y luego los angulos:

`m = (puntoFinal[1] - puntoInicial[1]) / (puntoFinal[0] - puntoInicial[0])`
`angulos = [m+(math.pi/2), m+(math.pi/2)+math.pi]`

> Con esto listo solo se dibujan los dos arcos y las dos lineas, posteriormente se les agrega las restricciones

`conList = []`

        conList.append(Sketcher.Constraint('Equal',contGeometria,contGeometria+1))
        conList.append(Sketcher.Constraint('Radius',contGeometria, radio))
        conList.append(Sketcher.Constraint('Tangent',contGeometria,1,contGeometria+2,1))
        conList.append(Sketcher.Constraint('Tangent',contGeometria,2,contGeometria+3,1))
        conList.append(Sketcher.Constraint('Tangent',contGeometria+1,1,contGeometria+3,2))
        conList.append(Sketcher.Constraint('Tangent',contGeometria+1,2,contGeometria+2,2))



In [17]:
nuevoSketch.crearRanura([2,1], [8,5], 2)

PuntoUno: 2.237462993461563, PuntoDos: 5.379055647051356


# Matriz Lineal
## Para utilizar la funcion de matriz lineal se debe seguir la siguiente estructura:

`s = doc.nuevoSketch("sketch", cuerpo, plano)`

`s.matrizLineal(elementos, colxfil, distancia, direccion = 0, pivote = None, clonar = False, acotar = False)`

> Como se puede observar es una funcion que puede recibir muchos parametros, sin embargo, no siempre se tienen que usar todos, como se puede observar tambien muchos de los parametros ya estan rellenos con un valor predeterminado con los valores que mas usualmente se usan, por ejemplo la direccion (0 = Horizontal), el punto pivote (None quiere decir que se utiliza el primer punto del primer elemento) y lo mismo con clonar y acotar, por lo que los argumentos minimos serian:

`s.matrizLineal(elementos, colxfil, distancia)`


## A continuacion se muestran ejemplos de uso con la geometria de un poligono:
`s.crearPoligono(10, [00,200] 5)`
![Geometria Base](./img/ejemploMatrizLineal01.png "Geometria Base")

1. ### Creacion de matriz Horizontal con relaciones de posicion respecto a la geometria original y acotaciones
`s.matrizLineal([0,1,2,3,4,5], 5, 25, clonar = True, acotar = True)`
![Resultado](./img/ejemploMatrizLineal02.png "Resultado Ejemplo 01")

2. ### Creacion de matriz con 25 grados de inclinacion
`s.matrizLineal([0,1,2,3,4,5], 5, 27.5, 25, clonar = True, acotar = True)`
![Resultado](./img/ejemploMatrizLineal03.png "Resultado Ejemplo 02")

3. ### Creacion de matriz de 3 filas y 3 columnas con 25 grados de inclinación y diferentes distancias entre filas y columnas
`s.matrizLineal([0,1,2,3,4,5], [3,3], [27.5, 40], 25, clonar = True, acotar = True)`
![Resultado](./img/ejemploMatrizLineal04.png "Resultado Ejemplo 03")